In [157]:
# !pip install wilds

In [158]:
import numpy as np
from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms
import torchvision.models as models
from wilds.common.data_loaders import get_eval_loader
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm

In [159]:
# Load the full dataset, and download it if necessary
dataset = get_dataset(dataset="camelyon17", download=False)

In [160]:
BATCH_SIZE = 32
FRACTION = 0.1

# Get the training set
train_data = dataset.get_subset(
    "train",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(train_data)) #302436 initially

"""
# (Optional) Load unlabeled data
dataset = get_dataset(dataset="camelyon17", download=True, unlabeled=True)
unlabeled_data = dataset.get_subset(
    "test_unlabeled",
    transform=transforms.Compose(
        [transforms.Resize((448, 448)), transforms.ToTensor()]
    ),
)
unlabeled_loader = get_train_loader("standard", unlabeled_data, batch_size=16)
"""
"""
# Train loop
for labeled_batch, unlabeled_batch in zip(train_loader, unlabeled_loader):
    x, y, metadata = labeled_batch
    unlabeled_x, unlabeled_metadata = unlabeled_batch
    ...
"""

30244


'\n# Train loop\nfor labeled_batch, unlabeled_batch in zip(train_loader, unlabeled_loader):\n    x, y, metadata = labeled_batch\n    unlabeled_x, unlabeled_metadata = unlabeled_batch\n    ...\n'

In [161]:
# Get the id_val set
id_val_data = dataset.get_subset(
    "id_val",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(id_val_data))

3356


In [162]:
# Get the val set
val_data = dataset.get_subset(
    "val",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(val_data))

3490


In [163]:
# Get the test set
test_data = dataset.get_subset(
    "test",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(test_data))


8505


In [164]:
resnet18_pretrained = models.resnet18(pretrained=True)

In [165]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 5

In [166]:
resnet18_pretrained.fc = nn.Linear(in_features=512, out_features=2, bias=True)
resnet18_pretrained.to(device)
resnet18_pretrained.load_state_dict(torch.load("resnet18_pretrained_all_grad.pt"))
def remove_classification_head(model):
    modules = list(model.children())[:-1]
    model = nn.Sequential(*modules)
    return model

resnet18_pretrained = remove_classification_head(resnet18_pretrained)

In [167]:
class SiameseNetworkDataset(Dataset):
    def __init__(self,dataset):
        self.data = dataset
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.data)

        #We need to approximately 50% of images to be in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #Look untill the same class image is found
                img1_tuple = random.choice(self.data) 
                if img0_tuple[1] == img1_tuple[1]:
                    break
        else:

            while True:
                #Look untill a different class image is found
                img1_tuple = random.choice(self.data) 
                if img0_tuple[1] != img1_tuple[1]:
                    break

        img0 = img0_tuple[0]
        img1 = img1_tuple[0]
        
        return img0, img1, torch.from_numpy(np.array([int(img1_tuple[1] != img0_tuple[1])], dtype=np.float32))
    
    def __len__(self):
        return len(self.data)

In [168]:
# X_train = [x[0].numpy() for x in train_data]
# y_train = [y[1].item() for y in train_data]


In [169]:
# print(X_train[:10])
 #print(y_train[:10])

In [170]:
import random

In [171]:
# print(np.where(np.asarray(y_train) == 0)[0])

In [172]:
"""
def make_pairs(x, y):
    Creates a tuple containing image pairs with corresponding label.

    Arguments:
        x: List containing images, each index in this list corresponds to one image.
        y: List containing labels, each label with datatype of `int`.

    Returns:
        Tuple containing two numpy arrays as (pairs_of_samples, labels),
        where pairs_of_samples' shape is (2len(x), 2,n_features_dims) and
        labels are a binary array of shape (2len(x)).
    
    # x = x.numpy()
    # y = y.numpy()
    num_classes = 2
    # [all indices where class=0, all ind where class=1 .... all ind where class=9]
    digit_indices = [np.where(np.asarray(y) == i)[0] for i in range(num_classes)]
    print(digit_indices)
    pairs = []
    labels = []

    for idx1 in range(len(x)):
        # add a matching example
        x1 = x[idx1]
        label1 = y[idx1]
        idx2 = random.choice(digit_indices[label1])
        x2 = x[idx2]

        pairs += [[x1, x2]]
        labels += [1]
        # add a non-matching example
        label2 = random.randint(0, num_classes - 1)
        while label2 == label1:
            label2 = random.randint(0, num_classes - 1)

        idx2 = random.choice(digit_indices[label2])
        x2 = x[idx2]

        pairs += [[x1, x2]]
        labels += [0]

    return np.array(pairs), np.array(labels).astype("float32")
"""

'\ndef make_pairs(x, y):\n    Creates a tuple containing image pairs with corresponding label.\n\n    Arguments:\n        x: List containing images, each index in this list corresponds to one image.\n        y: List containing labels, each label with datatype of `int`.\n\n    Returns:\n        Tuple containing two numpy arrays as (pairs_of_samples, labels),\n        where pairs_of_samples\' shape is (2len(x), 2,n_features_dims) and\n        labels are a binary array of shape (2len(x)).\n    \n    # x = x.numpy()\n    # y = y.numpy()\n    num_classes = 2\n    # [all indices where class=0, all ind where class=1 .... all ind where class=9]\n    digit_indices = [np.where(np.asarray(y) == i)[0] for i in range(num_classes)]\n    print(digit_indices)\n    pairs = []\n    labels = []\n\n    for idx1 in range(len(x)):\n        # add a matching example\n        x1 = x[idx1]\n        label1 = y[idx1]\n        idx2 = random.choice(digit_indices[label1])\n        x2 = x[idx2]\n\n        pairs += [[

In [173]:
# make train pairs
# pairs_train, labels_train = make_pairs(X_train, y_train)

In [174]:
# X_id_val = [x[0].numpy() for x in id_val_data]
# y_id_val = [y[1].item() for y in id_val_data]
# print(X_id_val[:10])
# print(y_id_val[:10])

In [175]:
# make validation pairs
# pairs_val, labels_val = make_pairs(X_id_val, y_id_val)

In [176]:
train_dataset = SiameseNetworkDataset(train_data)
id_val_dataset = SiameseNetworkDataset(id_val_data)
val_dataset = SiameseNetworkDataset(val_data)

In [177]:
#create the Siamese Neural Network
class SiameseNetwork(nn.Module):

    def __init__(self):
        super(SiameseNetwork, self).__init__()

        # Setting up the Sequential of CNN Layers
        self.embed = resnet18_pretrained

        # Setting up the Fully Connected Layers
        self.fc1 = nn.Sequential(            
            nn.Linear(512,2)
        )
        
    def forward_once(self, x):
        # This function will be called for both images
        # Its output is used to determine the similiarity
        output = self.embed(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        # In this function we pass in both images and obtain both vectors
        # which are returned
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)

        return output1, output2

In [178]:
# Define the Contrastive Loss Function
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
      # Calculate the euclidean distance and calculate the contrastive loss
      euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)

      loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                    (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


      return loss_contrastive

In [179]:
train_loader = DataLoader(train_dataset,
                        shuffle=True,
                        batch_size=BATCH_SIZE)

id_val_loader = DataLoader(id_val_dataset,
                        batch_size=BATCH_SIZE)

net = SiameseNetwork().to(device)

loss_criterion = ContrastiveLoss()

optimizer = torch.optim.SGD(resnet18_pretrained.parameters(), lr=0.01, momentum=0.9)

In [180]:
"""
counter = []
loss_history = [] 
iteration_number= 0

# Iterate throught the epochs
for epoch in range(num_epochs):

    # Iterate over batches
    for i, (img0, img1, label) in enumerate(train_dataloader, 0):

        # Send the images and labels to CUDA
        img0, img1, label = img0.cuda(), img1.cuda(), label.cuda()

        # Zero the gradients
        optimizer.zero_grad()

        # Pass in the two images into the network and obtain two outputs
        output1, output2 = net(img0, img1)

        # Pass the outputs of the networks and label into the loss function
        loss_contrastive = criterion(output1, output2, label)

        # Calculate the backpropagation
        loss_contrastive.backward()

        # Optimize
        optimizer.step()

        # Every 100 batches print out the loss
        if i % 100 == 0 :
            print(f"Epoch number {epoch}\n Current loss {loss_contrastive.item()}\n")
            iteration_number += 100

            counter.append(iteration_number)
            loss_history.append(loss_contrastive.item())
"""

'\ncounter = []\nloss_history = [] \niteration_number= 0\n\n# Iterate throught the epochs\nfor epoch in range(num_epochs):\n\n    # Iterate over batches\n    for i, (img0, img1, label) in enumerate(train_dataloader, 0):\n\n        # Send the images and labels to CUDA\n        img0, img1, label = img0.cuda(), img1.cuda(), label.cuda()\n\n        # Zero the gradients\n        optimizer.zero_grad()\n\n        # Pass in the two images into the network and obtain two outputs\n        output1, output2 = net(img0, img1)\n\n        # Pass the outputs of the networks and label into the loss function\n        loss_contrastive = criterion(output1, output2, label)\n\n        # Calculate the backpropagation\n        loss_contrastive.backward()\n\n        # Optimize\n        optimizer.step()\n\n        # Every 100 batches print out the loss\n        if i % 100 == 0 :\n            print(f"Epoch number {epoch}\n Current loss {loss_contrastive.item()}\n")\n            iteration_number += 100\n\n       

In [181]:
def train_epoch(model, train_dataloader, loss_crt, optimizer, device):
    """
    model: Model object
    train_dataloader: DataLoader over the training dataset
    loss_crt: loss function object
    optimizer: Optimizer object
    device: torch.device('cpu) or torch.device('cuda')

    The function returns:
     - the epoch training loss, which is an average over the individual batch
       losses
    """
    model.train()
    epoch_loss = 0.0

    num_batches = len(train_dataloader)
    # Iterate over batches
    for i, (img0, img1, label) in tqdm(enumerate(train_dataloader, 0)):

        # Send the images and labels to CUDA
        img0, img1, label = img0.to(device), img1.to(device), label.to(device)

        # Zero the gradients
        model.zero_grad()

        # Pass in the two images into the network and obtain two outputs
        output1, output2 = model(img0, img1)

        # Pass the outputs of the networks and label into the loss function
        loss_contrastive = loss_crt(output1, output2, label)

        epoch_loss += loss_contrastive.item()

        # Calculate the backpropagation
        loss_contrastive.backward()

        # Optimize
        optimizer.step()


    epoch_loss = epoch_loss/num_batches
    return epoch_loss

def eval_epoch(model, val_dataloader, loss_crt, device):
    """
    model: Model object
    val_dataloader: DataLoader over the validation dataset
    loss_crt: loss function object
    device: torch.device('cpu) or torch.device('cuda')

    The function returns:
     - the epoch validation loss, which is an average over the individual batch
       losses
    """
    model.eval()
    epoch_loss = 0.0

    num_batches = len(val_dataloader)
    with torch.no_grad():
      # Iterate over batches
      for i, (img0, img1, label) in tqdm(enumerate(val_dataloader, 0)):

          # Send the images and labels to CUDA
          img0, img1, label = img0.to(device), img1.to(device), label.to(device)

          # Pass in the two images into the network and obtain two outputs
          output1, output2 = model(img0, img1)

          # Pass the outputs of the networks and label into the loss function
          loss_contrastive = loss_crt(output1, output2, label)

          epoch_loss += loss_contrastive.item()

    epoch_loss = epoch_loss/num_batches
  
    return epoch_loss

In [182]:
train_losses = []
train_accuracies = []
id_val_losses = []
id_val_accuracies = []
for epoch in range(1, num_epochs+1):
  train_loss = train_epoch(net, train_loader, loss_criterion, optimizer, device)
  val_loss = eval_epoch(net, id_val_loader, loss_criterion, device)
  train_losses.append(train_loss)
  id_val_losses.append(val_loss)
  print('\nEpoch %d'%(epoch))
  print('train loss: %10.8f'%(train_loss))
  print('id_val loss: %10.8f'%(val_loss))

946it [03:29,  4.53it/s]
105it [00:24,  4.22it/s]



Epoch 1
train loss: 0.08925023
id_val loss: 0.08437311


946it [03:11,  4.95it/s]
105it [00:20,  5.12it/s]



Epoch 2
train loss: 0.03654778
id_val loss: 0.13190774


946it [03:09,  4.99it/s]
105it [00:20,  5.11it/s]



Epoch 3
train loss: 0.02860975
id_val loss: 0.15463225


946it [03:08,  5.01it/s]
105it [00:20,  5.07it/s]



Epoch 4
train loss: 0.02391434
id_val loss: 0.12014466


946it [03:08,  5.01it/s]
105it [00:22,  4.77it/s]


Epoch 5
train loss: 0.02135456
id_val loss: 0.07285507


In [213]:
val_loader = DataLoader(val_dataset,
                        batch_size=BATCH_SIZE)

In [217]:
import torchvision
from matplotlib.pyplot import imshow
import torchvision.utils
# Grab one image that we are going to test
dataiter = iter(id_val_loader)
x0, _, label1 = next(dataiter)

dissimilarity = [[], []]

for i in range(5):
    # Iterate over 5 images and test them with the first image (x0)
    _, x1, label2 = next(dataiter)
    
    output1, output2 = net(x0.cuda(), x1.cuda())

    euclidean_distance = F.pairwise_distance(output1, output2)
    """
    talking on the phone
    """
    for idx in range(BATCH_SIZE):
      print(label1[idx].item(), label2[idx].item())
      print(f'Dissimilarity: {euclidean_distance[idx].item():.2f}')
      print("\n")
      dissimilarity[1 if label1[idx].item() == label2[idx].item() else 0].append(euclidean_distance[idx].item())




0.0 1.0
Dissimilarity: 0.02


0.0 0.0
Dissimilarity: 2.06


1.0 0.0
Dissimilarity: 2.03


0.0 1.0
Dissimilarity: 0.01


1.0 1.0
Dissimilarity: 2.06


1.0 0.0
Dissimilarity: 2.08


0.0 0.0
Dissimilarity: 2.10


0.0 0.0
Dissimilarity: 0.02


1.0 1.0
Dissimilarity: 0.02


1.0 0.0
Dissimilarity: 0.00


1.0 1.0
Dissimilarity: 0.02


1.0 0.0
Dissimilarity: 1.90


0.0 0.0
Dissimilarity: 0.02


1.0 0.0
Dissimilarity: 0.39


0.0 1.0
Dissimilarity: 0.07


1.0 0.0
Dissimilarity: 2.15


0.0 0.0
Dissimilarity: 2.09


0.0 1.0
Dissimilarity: 0.02


1.0 0.0
Dissimilarity: 0.03


1.0 0.0
Dissimilarity: 0.01


1.0 1.0
Dissimilarity: 0.02


1.0 1.0
Dissimilarity: 2.04


0.0 0.0
Dissimilarity: 2.09


1.0 1.0
Dissimilarity: 0.02


1.0 1.0
Dissimilarity: 1.21


1.0 1.0
Dissimilarity: 0.01


1.0 1.0
Dissimilarity: 0.04


1.0 0.0
Dissimilarity: 2.04


1.0 0.0
Dissimilarity: 0.08


0.0 0.0
Dissimilarity: 0.02


0.0 1.0
Dissimilarity: 1.96


1.0 0.0
Dissimilarity: 0.07


0.0 1.0
Dissimilarity: 0.02


0.0 1.0
Di

In [218]:
import numpy as np
def calc_mean_std(arr):
    arr = np.asarray(arr)
    return arr.mean(), arr.std()


print(calc_mean_std(dissimilarity[0]))
print(calc_mean_std(dissimilarity[1]))

(1.1635585316781591, 0.969634864758567)
(1.0542280314278156, 0.9883026254441165)


In [219]:
dissimilarity = [[], []]
dataiter = iter(val_loader)
for i in range(5):
    # Iterate over 5 images and test them with the first image (x0)
    _, x1, label2 = next(dataiter)
    
    output1, output2 = net(x0.cuda(), x1.cuda())

    euclidean_distance = F.pairwise_distance(output1, output2)
    """
    talking on the phone
    """
    for idx in range(BATCH_SIZE):
      print(label1[idx].item(), label2[idx].item())
      print(f'Dissimilarity: {euclidean_distance[idx].item():.2f}')
      print("\n")
      dissimilarity[1 if label1[idx].item() == label2[idx].item() else 0].append(euclidean_distance[idx].item())

0.0 1.0
Dissimilarity: 0.02


0.0 1.0
Dissimilarity: 2.09


1.0 0.0
Dissimilarity: 1.95


0.0 0.0
Dissimilarity: 0.09


1.0 0.0
Dissimilarity: 2.07


1.0 0.0
Dissimilarity: 0.43


0.0 1.0
Dissimilarity: 0.02


0.0 1.0
Dissimilarity: 0.04


1.0 1.0
Dissimilarity: 0.01


1.0 0.0
Dissimilarity: 2.15


1.0 0.0
Dissimilarity: 1.12


1.0 0.0
Dissimilarity: 1.50


0.0 0.0
Dissimilarity: 2.00


1.0 0.0
Dissimilarity: 2.10


0.0 0.0
Dissimilarity: 0.58


1.0 0.0
Dissimilarity: 0.48


0.0 0.0
Dissimilarity: 1.98


0.0 1.0
Dissimilarity: 2.08


1.0 1.0
Dissimilarity: 0.31


1.0 0.0
Dissimilarity: 2.16


1.0 1.0
Dissimilarity: 0.01


1.0 1.0
Dissimilarity: 0.02


0.0 0.0
Dissimilarity: 2.12


1.0 0.0
Dissimilarity: 1.18


1.0 1.0
Dissimilarity: 0.95


1.0 0.0
Dissimilarity: 0.12


1.0 1.0
Dissimilarity: 2.07


1.0 0.0
Dissimilarity: 1.90


1.0 1.0
Dissimilarity: 2.10


0.0 1.0
Dissimilarity: 2.05


0.0 1.0
Dissimilarity: 0.63


1.0 1.0
Dissimilarity: 0.03


0.0 1.0
Dissimilarity: 2.02


0.0 1.0
Di

In [220]:
print(calc_mean_std(dissimilarity[0]))
print(calc_mean_std(dissimilarity[1]))

(1.1980879493744279, 0.9067255947178056)
(0.9520744159306604, 0.9133478666747794)
